<h1 style="background-color:#0071BD;color:white;text-align:center;padding-top:0.8em;padding-bottom: 0.8em">
  LDA Spike 0 - Create a Corpus of Answers on abgeordnetenwatch.de
</h1>

Creation and update of a corpus of answers on abgeordnetenwatch.de. See

  * Information about the deputies of the Bundestag: https://www.abgeordnetenwatch.de/bundestag
  * Application Programming interfact to access the data: https://www.abgeordnetenwatch.de/api
  
<p style="background-color:#66A5D1;padding-top:0.2em;padding-bottom: 0.2em" />

In [1]:
import json
import requests

from pathlib import Path
from lxml import html

In [2]:
# Read stored values of configuration parameters or set a default

%store -r proxies
if not('proxies' in globals()): proxies = {}

%store -r project_name
if not('project_name' in globals()): project_name = 'AbgeordnetenWatch'

%store -r text_data_dir
if not('text_data_dir' in globals()): text_data_dir = Path.home() / 'TextData'

In [3]:
# Set these variables to False to force an update on all deputies of all known questions
# To force an update on a deputy otherwise, just delete the json file.
# Additional questions are only recognized if the deputy gets updated.

update_only_missing_deputies = True
update_only_missing_answers = True

In [4]:
corpus_dir = text_data_dir / project_name / 'Corpus'
corpus_dir.mkdir(parents=True, exist_ok=True) # Creates a local directory!

## Utility function guessing the schema from given JSON data

In [5]:
def print_json_schema(json, indentation=-1):

    if isinstance(json, dict):
        print()
        indentation += 1
        for key, value in json.items():
            print(indentation * '    ', key, end=': ')
            print_json_schema(value, indentation)
        indentation -= 1
        print()
    
    elif isinstance(json, list):
        length = len(json)
        if length:
            print(length, end='x ')
            print_json_schema(json[0], indentation)
        else:
            print('0x ???')
    
    else:
        print(type(json))

In [6]:
print_json_schema([{'first': 'Angela', 'middle': 'Dorothea ', 'last': 'Merkel'}, {'first': 'Emmanuel', 'last': 'Macron'}])

2x 
 first: <class 'str'>
 middle: <class 'str'>
 last: <class 'str'>



In [7]:
print_json_schema([{'first': 'Emmanuel', 'last': 'Macron'}, {'first': 'Angela', 'middle': 'Dorothea ', 'last': 'Merkel'}])

2x 
 first: <class 'str'>
 last: <class 'str'>



In [8]:
print_json_schema({'countries': ['Germany', 'France'], 'politicians': [{'first': 'Angela', 'last': 'Merkel'}, {}, {}]})


 countries: 2x <class 'str'>
 politicians: 3x 
     first: <class 'str'>
     last: <class 'str'>




## Retrieve the list of parliaments

In [9]:
parliaments_url  = 'https://www.abgeordnetenwatch.de/api/parliaments.json'
parliaments_json = requests.get(parliaments_url, proxies=proxies).json() # Request to abgeordnetenwatch.de!

In [10]:
print_json_schema(parliaments_json)


 meta: 
     license: 
         name: <class 'str'>
         url: <class 'str'>

     contributer: 0x ???
     subsets: 0x ???

 parliaments: 78x 
     name: <class 'str'>
     meta: 
         uuid: <class 'str'>

     dates: 
         start: <class 'str'>
         end: <class 'str'>
         election: <class 'str'>

     datasets: 
         deputies: 
             by-name: <class 'str'>
             by-uuid: <class 'str'>

         candidates: 
             by-name: <class 'str'>
             by-uuid: <class 'str'>

         constituencies: 
             by-name: <class 'str'>
             by-uuid: <class 'str'>

         polls: 
             by-name: <class 'str'>
             by-uuid: <class 'str'>

         committees: 
             by-name: <class 'str'>
             by-uuid: <class 'str'>






In [11]:
parliaments = parliaments_json['parliaments']

for parliament in parliaments[:19]:
    print(parliament['name'], end=', ')

Baden-Württemberg, Baden-Württemberg 2006-2011, Baden-Württemberg 2011-2016, Bayern, Bayern 2008-2013, Bayern 2013-2018, Berlin, Berlin 2006-2011, Berlin 2011-2016, Brandenburg, Brandenburg 2009-2014, Bremen, Bremen 2007-2011, Bremen 2011-2015, Bundestag, Bundestag 2005-2009, Bundestag 2009-2013, Bundestag 2013-2017, Bürgermeisterwahlen Nordrhein-Westfalen 2009, 

### Example: Bundestag

In [12]:
search_parliament_name = 'Bundestag'

parliament = next(p for p in parliaments if p['name'] == search_parliament_name)

deputies_url = parliament['datasets']['deputies']['by-name']
parliament_name = deputies_url.split('/')[-2]

print('Parliament  :', parliament['name'])
print('  in URLs   :', parliament_name)
print('Dates       :', parliament['dates'])
print('Deputies URL:', deputies_url)

Parliament  : Bundestag
  in URLs   : bundestag
Dates       : {'start': '2017-07-20', 'end': '2021-10-23', 'election': '2017-09-24'}
Deputies URL: https://www.abgeordnetenwatch.de/api/parliament/bundestag/deputies.json


## Retrieve the list of all deputies of the Bundestag

In [13]:
print(deputies_url)

https://www.abgeordnetenwatch.de/api/parliament/bundestag/deputies.json


In [14]:
deputies_json = requests.get(deputies_url, proxies=proxies).json() # Request to abgeordnetenwatch.de!

In [15]:
print_json_schema(deputies_json)


 meta: 
     license: 
         name: <class 'str'>
         url: <class 'str'>

     contributer: 1x <class 'str'>
     subsets: 3x <class 'str'>

 profiles: 709x 
     meta: 
         status: <class 'str'>
         edited: <class 'str'>
         uuid: <class 'str'>
         username: <class 'str'>
         questions: <class 'int'>
         answers: <class 'int'>
         standard_replies: <class 'int'>
         url: <class 'str'>

     personal: 
         degree: <class 'NoneType'>
         first_name: <class 'str'>
         last_name: <class 'str'>
         gender: <class 'str'>
         birthyear: <class 'str'>
         education: <class 'str'>
         profession: <class 'str'>
         location: 
             country: <class 'str'>
             state: <class 'str'>
             city: <class 'str'>
             postal_code: <class 'str'>

         picture: 
             url: <class 'str'>
             copyright: <class 'str'>


     party: <class 'str'>
     parliament: 
        

In [16]:
deputies = deputies_json ['profiles']

for deputy in deputies[:23]:
    print(deputy['meta']['username'], end=', ')

detlef-seif, dirk-wiese, dirk-vopel, kersten-steinke, ursula-schulte, axel-schafer, susann-ruthrich, johannes-roring, rene-rospel, swen-schulz, thomas-rachel, alois-rainer, achim-post, henning-otte, dr-andreas-nick, friedrich-ostendorff, susanne-mittag, sabine-leidig, katharina-landgraf, roy-kuhne, chris-kuhn, astrid-groteluschen, sabine-poschmann, 

### Example: "Ulrich Kelber" in the response about all deputies

In [17]:
# API URL for a deputy file based on the structure of parliament['datasets']['deputies']['by-name'] and 
# the example https://www.abgeordnetenwatch.de/api/parliament/bundestag/profile/angela-merkel/profile.json
# as given on https://www.abgeordnetenwatch.de/api

def deputy_api_url(deputies_url, deputy_name):
    return '/'.join(deputies_url.split('/')[:-1] + ['profile', deputy_name, 'profile.json'] )

In [18]:
search_first_name = 'Ulrich'
search_last_name  = 'Kelber'

deputy = next(a for a in deputies 
                  if a['personal']['first_name'] == search_first_name 
                      and a['personal']['last_name']  == search_last_name)

deputy_name = deputy['meta']['username']
deputy_url = deputy_api_url(deputies_url, deputy_name)

print('Deputy       :', deputy['personal']['first_name'], deputy['personal']['last_name'], '('+deputy['party']+')')
print('  in URLs    :', deputy_name)
print('Profile URL  :', deputy['meta']['url'])
print('  API URL    :', deputy_url)
print('Year of birth:', deputy['personal']['birthyear'])
print('Education    :', deputy['personal']['education'])
print('Election     :', deputy['constituency']['result'] + '%', 'in', deputy['constituency']['name'])
for i, committee in enumerate(deputy['committees']):
    print('Commitee {}   :'.format(i), committee['name'])
print('Answers      :', deputy['meta']['answers'], 'regular,', deputy['meta']['standard_replies'], 
      'standard, for', deputy['meta']['questions'], 'questions')

Deputy       : Ulrich Kelber (SPD)
  in URLs    : ulrich-wolfgang-kelber
Profile URL  : https://www.abgeordnetenwatch.de/profile/ulrich-wolfgang-kelber
  API URL    : https://www.abgeordnetenwatch.de/api/parliament/bundestag/profile/ulrich-wolfgang-kelber/profile.json
Year of birth: 1968
Education    : Diplom-Informatiker
Election     : 34,9% in Bonn
Commitee 0   : Ausschuss Digitale Agenda
Answers      : 18 regular, 0 standard, for 18 questions


### Example: Questions to "Ulrich Kelber" listed in his profile

In [19]:
print(deputy_url)

https://www.abgeordnetenwatch.de/api/parliament/bundestag/profile/ulrich-wolfgang-kelber/profile.json


In [20]:
deputy_json = requests.get(deputy_url, proxies=proxies).json() # Request to abgeordnetenwatch.de!

In [21]:
print_json_schema(deputy_json)


 profile: 
     meta: 
         status: <class 'str'>
         edited: <class 'str'>
         uuid: <class 'str'>
         username: <class 'str'>
         questions: <class 'int'>
         answers: <class 'int'>
         standard_replies: <class 'int'>
         url: <class 'str'>

     personal: 
         degree: <class 'NoneType'>
         first_name: <class 'str'>
         last_name: <class 'str'>
         gender: <class 'str'>
         birthyear: <class 'str'>
         education: <class 'str'>
         profession: <class 'str'>
         location: 
             country: <class 'str'>
             state: <class 'str'>
             city: <class 'str'>
             postal_code: <class 'str'>

         picture: 
             url: <class 'str'>
             copyright: <class 'str'>


     party: <class 'str'>
     parliament: 
         name: <class 'str'>
         uuid: <class 'str'>

     roles: 0x ???
     constituency: 
         name: <class 'str'>
         uuid: <class 'str'>
      

In [22]:
questions = deputy_json['profile']['questions'] 

for question in questions[:11]:
    print(question['date'], question['category'], len(question['answers']), end = ', ')

2018-12-05 Demokratie und Bürgerrechte 1, 2018-11-02 Demokratie und Bürgerrechte 1, 2018-09-25 Umwelt 1, 2018-09-20 Umwelt 1, 2018-09-14 Umwelt 1, 2018-08-23 Gesundheit 1, 2018-07-24 Gesundheit 1, 2018-07-24 Demokratie und Bürgerrechte 1, 2018-06-26 Internationales 1, 2018-06-18 Land- und Forstwirtschaft 1, 2018-06-11 Finanzen 1, 

## Utility functions for naming files and extracting text from HTML


In [23]:
def deputy_file_name_part(deputy):
    return '_'.join([deputy['meta']['username'], deputy['party'].lower().replace(' ', '-')])
                     
def question_file_name_parts(q, question):
    question_nr = 'Q{:04}'.format(q + 1) # Maximum in 12/18: 344 questions (Andrea Nahles)
    question_id = '_'.join([question_nr, question['date']])
    category    = question['category'].lower().replace(' ', '-')
    return question_id, category
    
def answer_file_name_part(a, answer):
    answer_nr   = 'A{:02}'.format(a + 1) # Maximum in 12/18: 2 answer for one question (often)
    return '_'.join([answer_nr, answer['date']])

In [24]:
print(deputy_file_name_part(deputy))

if questions:
    oldest = len(questions) - 1
    print(question_file_name_parts(oldest, questions[oldest]))
    
    answers = questions[oldest]['answers']
    if answers:
        print(answer_file_name_part(0, answers[0]))

ulrich-wolfgang-kelber_spd
('Q0036_2017-07-24', 'inneres-und-justiz')
A01_2017-07-25


In [25]:
def extract_answers_as_text(html_text):

    page = html.fromstring(html_text)
    for nocontent in page.find_class('robots-nocontent'): nocontent.clear()
    
    answers  = page.find_class('question__answer')

    # Names of citizens asking a question are "encrypted" but still unique.
    # To keep them even more private, we replace all these names by 'N.N.'.
    for answer in answers:
        for name in answer.find_class('crypto-font'): name.text = ' N.N. '
    
    for author in page.find_class('question__question__author'): author.clear()
    for author in page.find_class('question__answer__author'): author.clear()
    
    return [answer.text_content() for answer in answers]

In [26]:
encrypted_name = 'Hijklmn'

html_text = '''
<!DOCTYPE html>
<html lang="de" dir="ltr">
  <head></head>
  <body>
    <main id="content">

<div class="container-small">
  <div class="question__question__title">    
    <p>... Antrag der FDP gestimmt, ... dass Deutschland eine Abschaffung der Sommerzeit wünscht ... </p>
  </div>
    <p class="question__question__author">Von: 
      <span class="robots-nocontent">
        <span class="crypto-font">Abcdefg Hijklmn</span>
      </span>
    </p>
</div>

<div class="question__answer-wrapper">
  <div class="question__answer">
    <p class="question__answer__author">
      Antwort von <strong>Ulrich Kelber (SPD)</strong>
      <span>26. März. 2018 - 14:48<br>
        <small>Dauer bis zur Antwort: 1 Tag 6 Stunden</small>
      </span>
    </p>
    <p>Sehr geehrter Herr <span class="crypto-font">Hijklmn</span>,</p>
    <p>vielen Dank für Ihre Anfrage zur Sommerzeit.<br /> Ich denke, ...</p>
    <p>Mit freundlichem Gruß <br />Ulrich Kelber</p>      
  </div>
</div>

    </main>
  </body>
</html>
'''

print('Original HTML contains', 'a' if encrypted_name in html_text else 'no', 'reference to the encrypted name.')
for a, answer_text in enumerate(extract_answers_as_text(html_text)):
    print('Answer {}:'.format(a+1))
    for line in answer_text.split('\n'):
        print(line)
    print('Extracted answer contains', 'a' if encrypted_name in answer_text else 'no', 'reference to the encrypted name.')

Original HTML contains a reference to the encrypted name.
Answer 1:

    Sehr geehrter Herr  N.N. ,
    vielen Dank für Ihre Anfrage zur Sommerzeit. Ich denke, ...
    Mit freundlichem Gruß Ulrich Kelber      
  
Extracted answer contains no reference to the encrypted name.


## Create corpus of all answers of all deputies of the Bundestag

### Create or upate deputy files (JSON) and question files (URL)

In [27]:
success = []
failure = []

for d, deputy in enumerate(deputies):

    deputy_prefix = deputy_file_name_part(deputy)
    deputy_file = corpus_dir / (deputy_prefix + '.json')

    try:
        if update_only_missing_deputies and deputy_file.exists(): continue

        deputy_url = deputy_api_url(deputies_url, deputy['meta']['username'])
        deputy_json = requests.get(deputy_url, proxies=proxies).json() # Request to abgeordnetenwatch.de!
        deputy_file.write_text(json.dumps(deputy_json))
        success.append(deputy_file.name)
        
        questions = deputy_json['profile']['questions']
        for q, question in enumerate(reversed(questions)): # Oldest question first

            question_infix, question_suffix = question_file_name_parts(q, question)
            url_filename = '_'.join([deputy_prefix, question_infix, question_suffix]) + '.url'
            url_file = corpus_dir / url_filename
            url_file.write_text(question['url'])
            success.append(url_file.name)   
        
    except Exception as exception:
        failure.append(deputy_file.name, exception)

    finally:
        print('\r{} of {}. {} files successfully created. {} files failed. Latest: {:30.30}'.format(
                 d+1, len(deputies), len(success), len(failure), deputy_file.stem), end='')

173 of 709. 0 files successfully created. 0 files failed. Latest: dr-reinhard-brandl_csu        

311 of 709. 0 files successfully created. 0 files failed. Latest: florian-osner_csu             

701 of 709. 0 files successfully created. 0 files failed. Latest: sybille-benning_cdu           

709 of 709. 0 files successfully created. 0 files failed. Latest: britta-katharina-dassler_fdp  

In [28]:
for deputy_filename, exception in failure:
    print('Exception while processing deputy {}:'.format(deputy_filename))
    print(exception)
    print()

if not(failure):
    print('No exception while updating the deputies and questions :-)')
    print()

print('{} files created or updated:'.format(len(success)))
print(', '.join(success))

No exception while updating the deputies and questions :-)

0 files created or updated:



### Create or upate answer files (TXT)

In [29]:
success = []
failure = []

for d, deputy in enumerate(deputies):

    deputy_prefix = deputy_file_name_part(deputy)
    deputy_file = corpus_dir / (deputy_prefix + '.json')
    
    questions = json.loads(deputy_file.read_text())['profile']['questions']
    
    for q, question in enumerate(reversed(questions)): # Oldest question first

        a = -1
        try:
            question_infix, question_suffix = question_file_name_parts(q, question)
            answer_files = []

            answers = question['answers']
            for a, answer in enumerate(answers):
                answer_infix = answer_file_name_part(a, answer)
                answer_filename = '_'.join([deputy_prefix, question_infix, answer_infix, question_suffix]) + '.txt'
                answer_files.append(corpus_dir / answer_filename)
                
            if update_only_missing_answers and all(file.exists() for file in answer_files): continue
                
            question_page = requests.get(question['url'], proxies=proxies).text # Request to abgeordnetenwatch.de!

            answer_texts = extract_answers_as_text(question_page)
                
            for file, text in zip(answer_files, answer_texts):
                if update_only_missing_answers and file.exists(): continue
                file.write_text(text)
                success.append(file.name)
            
        except Exception as exception:
             failure.append((deputy_prefix, q, a, exception))

        finally:
             print('\rDeputy {} of {}. Question {} of {}. {} files created. {} files failed. Latest: {:30.30}'.format(
                 d+1, len(deputies), q+1, len(questions), len(success), len(failure), deputy_prefix), end='')

Deputy 31 of 709. Question 6 of 7. 0 files created. 0 files failed. Latest: frank-junge_spd               

Deputy 709 of 709. Question 5 of 5. 0 files created. 0 files failed. Latest: britta-katharina-dassler_fdp      

In [30]:
for deputy_prefix, q, a, exception in failure:
    print('Exception while processing answer {} for question {} for deputy {}:'.format(a+1, q+1, deputy_prefix))
    print(exception)
    print()

if not(failure):
    print('No exception while updating the answers :-)')
    print()
    
print('{} files created or updated:'.format(len(success)))
print(', '.join(success))    

No exception while updating the answers :-)

0 files created or updated:



### Review of the corpus: counts, answers without questions, questions without answers

In [31]:
print(len(list(corpus_dir.glob('*.json'))), 'deputies')
print(len(list(corpus_dir.glob('*.url'))), 'questions')
print(len(list(corpus_dir.glob('*.txt'))), 'answers')
print(len(list(corpus_dir.glob('*A02*.txt'))), 'questions have two answers or more')

709 deputies
9228 questions
7394 answers
49 questions have two answers or more


In [32]:
def unique_filename_parts(pattern, name_slice):
    files = list(corpus_dir.glob(pattern))
    parts = sorted(['_'.join(f.stem.split('_')[name_slice]) for f in files])
    return parts

for d, deputy in enumerate(deputies):

    deputy_prefix = deputy_file_name_part(deputy)
    print('{} of {}, {:50.50}'.format(d+1, len(deputies), deputy_prefix), end='\r')
    
    questions = unique_filename_parts(deputy_prefix + '*.url', slice(4))
    answered  = unique_filename_parts(deputy_prefix + '*.txt', slice(4))

    answer_without_question = [q for q in answered if not q in questions]
    unanswered_questions = [q for q in questions if not q in answered]

    if answer_without_question or unanswered_questions:
        print('\n')
        
    if answer_without_question:
        print('Following questions are answered, but the question is not known itself:')
        print(', '.join(answer_without_question))
        print()
        
    if unanswered_questions:
        print('Following questions are not answered:')
        print(', '.join(unanswered_questions))
        print()

8 of 709, johannes-roring_cdu                               

Following questions are not answered:
johannes-roring_cdu_Q0003_2018-07-23

13 of 709, achim-post_spd                                    

Following questions are not answered:
achim-post_spd_Q0002_2017-08-10

14 of 709, henning-otte_cdu                                  

Following questions are not answered:
henning-otte_cdu_Q0011_2018-07-14

20 of 709, roy-kuhne_cdu                                     

Following questions are not answered:
roy-kuhne_cdu_Q0002_2017-09-10, roy-kuhne_cdu_Q0003_2017-09-20

22 of 709, astrid-groteluschen_cdu                           

Following questions are not answered:
astrid-groteluschen_cdu_Q0004_2017-09-14, astrid-groteluschen_cdu_Q0005_2017-10-22

28 of 709, michael-donth_cdu                                 

Following questions are not answered:
michael-donth_cdu_Q0001_2017-08-02, michael-donth_cdu_Q0002_2017-08-02, michael-donth_cdu_Q0003_2017-08-02, michael-donth_cdu_Q0004_2017-08-2

84 of 709, jens-lehmann_cdu                                  

Following questions are not answered:
jens-lehmann_cdu_Q0001_2017-08-05, jens-lehmann_cdu_Q0002_2017-08-17, jens-lehmann_cdu_Q0003_2017-08-21, jens-lehmann_cdu_Q0004_2017-09-19

86 of 709, birgit-malsack-winkemann_afd                      

Following questions are not answered:
birgit-malsack-winkemann_afd_Q0001_2017-12-07

88 of 709, christoph-neumann_afd                             

Following questions are not answered:
christoph-neumann_afd_Q0001_2017-08-17, christoph-neumann_afd_Q0002_2017-09-19, christoph-neumann_afd_Q0003_2017-09-21, christoph-neumann_afd_Q0004_2018-06-24

89 of 709, martin-reichardt_afd                              

Following questions are not answered:
martin-reichardt_afd_Q0001_2018-01-30

91 of 709, ingrid-lieselotte-remmers_die-linke               

Following questions are not answered:
ingrid-lieselotte-remmers_die-linke_Q0001_2018-07-27

92 of 709, bernd-riexinger_die-linke                   

138 of 709, michael-gros_spd                                  

Following questions are not answered:
michael-gros_spd_Q0007_2017-09-19, michael-gros_spd_Q0010_2018-09-04

139 of 709, alexander-ulrich_die-linke                        

Following questions are not answered:
alexander-ulrich_die-linke_Q0006_2017-12-14

140 of 709, hans-joachim-fuchtel_cdu                          

Following questions are not answered:
hans-joachim-fuchtel_cdu_Q0001_2017-08-02, hans-joachim-fuchtel_cdu_Q0002_2017-08-23, hans-joachim-fuchtel_cdu_Q0004_2017-12-13, hans-joachim-fuchtel_cdu_Q0005_2018-01-19, hans-joachim-fuchtel_cdu_Q0006_2018-09-05

141 of 709, yvonne-magwas_cdu                                 

Following questions are not answered:
yvonne-magwas_cdu_Q0008_2017-08-22, yvonne-magwas_cdu_Q0011_2017-09-12, yvonne-magwas_cdu_Q0012_2018-04-27

142 of 709, michelle-muntefering_spd                          

Following questions are not answered:
michelle-muntefering_spd_Q0006_2017-09-23, michelle-

202 of 709, marcus-held_spd                                   

Following questions are not answered:
marcus-held_spd_Q0008_2017-09-15

203 of 709, konstantin-kuhle_fdp                              

Following questions are not answered:
konstantin-kuhle_fdp_Q0001_2017-07-25, konstantin-kuhle_fdp_Q0002_2017-08-18, konstantin-kuhle_fdp_Q0004_2018-09-13

204 of 709, waldemar-herdt_afd                                

Following questions are not answered:
waldemar-herdt_afd_Q0001_2017-09-04, waldemar-herdt_afd_Q0004_2017-10-11, waldemar-herdt_afd_Q0005_2017-10-20, waldemar-herdt_afd_Q0006_2018-06-05

206 of 709, dr-bettina-hoffmann_die-grünen                    

Following questions are not answered:
dr-bettina-hoffmann_die-grünen_Q0003_2017-08-02, dr-bettina-hoffmann_die-grünen_Q0008_2018-10-09

207 of 709, christian-sauter_fdp                              

Following questions are not answered:
christian-sauter_fdp_Q0001_2017-08-03, christian-sauter_fdp_Q0002_2017-08-20, christian-saute

246 of 709, katja-suding_fdp                                  

Following questions are not answered:
katja-suding_fdp_Q0009_2017-09-14

247 of 709, veronika-bellmann_cdu                             

Following questions are not answered:
veronika-bellmann_cdu_Q0002_2017-10-29, veronika-bellmann_cdu_Q0003_2018-03-06, veronika-bellmann_cdu_Q0004_2018-09-08, veronika-bellmann_cdu_Q0005_2018-10-29

248 of 709, erhard-grundl_die-grünen                          

Following questions are not answered:
erhard-grundl_die-grünen_Q0014_2018-06-13, erhard-grundl_die-grünen_Q0015_2018-09-11, erhard-grundl_die-grünen_Q0016_2018-10-29

249 of 709, johannes-selle_cdu                                

Following questions are not answered:
johannes-selle_cdu_Q0012_2018-10-30

250 of 709, andreas-mattfeldt_cdu                             

Following questions are not answered:
andreas-mattfeldt_cdu_Q0004_2018-04-22, andreas-mattfeldt_cdu_Q0006_2018-11-02

251 of 709, dr-karamba-diaby_spd                 

...

546 of 709, erwin-ruddel_cdu                                  

Following questions are not answered:
erwin-ruddel_cdu_Q0020_2018-12-10

547 of 709, nicole-westig_fdp                                 

Following questions are not answered:
nicole-westig_fdp_Q0002_2017-08-02, nicole-westig_fdp_Q0003_2017-08-31, nicole-westig_fdp_Q0004_2017-09-08

549 of 709, dr-fritz-felgentreu_spd                           

Following questions are not answered:
dr-fritz-felgentreu_spd_Q0035_2018-11-17, dr-fritz-felgentreu_spd_Q0036_2018-12-12

551 of 709, hilde-mattheis_spd                                

Following questions are not answered:
hilde-mattheis_spd_Q0003_2017-09-13, hilde-mattheis_spd_Q0004_2017-09-19

554 of 709, dr-johannes-fechner_spd                           

Following questions are not answered:
dr-johannes-fechner_spd_Q0002_2017-09-03

555 of 709, udo-hemmelgarn_afd                                

Following questions are not answered:
udo-hemmelgarn_afd_Q0003_2018-12-13

556 of 7

599 of 709, stefan-schwartze_spd                              

Following questions are not answered:
stefan-schwartze_spd_Q0012_2018-12-14

602 of 709, frank-schwabe_spd                                 

Following questions are not answered:
frank-schwabe_spd_Q0004_2017-09-20

603 of 709, michael-frieser_csu                               

Following questions are not answered:
michael-frieser_csu_Q0010_2018-06-25, michael-frieser_csu_Q0016_2018-12-15

604 of 709, michael-grosse-bromer-2_cdu                       

Following questions are not answered:
michael-grosse-bromer-2_cdu_Q0015_2018-09-21, michael-grosse-bromer-2_cdu_Q0018_2018-12-05, michael-grosse-bromer-2_cdu_Q0019_2018-12-14

605 of 709, svenja-stadler_spd                                

Following questions are not answered:
svenja-stadler_spd_Q0023_2018-10-11, svenja-stadler_spd_Q0027_2018-12-14

606 of 709, eckhard-pols_cdu                                  

Following questions are not answered:
eckhard-pols_cdu_Q0017_20

641 of 709, jens-spahn_cdu                                    

Following questions are not answered:
jens-spahn_cdu_Q0044_2018-04-20, jens-spahn_cdu_Q0051_2018-06-28, jens-spahn_cdu_Q0052_2018-07-15, jens-spahn_cdu_Q0054_2018-07-21, jens-spahn_cdu_Q0060_2018-08-13, jens-spahn_cdu_Q0061_2018-08-21, jens-spahn_cdu_Q0062_2018-08-23, jens-spahn_cdu_Q0072_2018-10-30

643 of 709, gero-storjohann_cdu                               

Following questions are not answered:
gero-storjohann_cdu_Q0008_2018-11-11, gero-storjohann_cdu_Q0009_2018-12-16

644 of 709, karin-maag_cdu                                    

Following questions are not answered:
karin-maag_cdu_Q0005_2017-08-02, karin-maag_cdu_Q0007_2017-08-09, karin-maag_cdu_Q0008_2017-08-13, karin-maag_cdu_Q0011_2018-06-27, karin-maag_cdu_Q0013_2018-12-11, karin-maag_cdu_Q0014_2018-12-15

645 of 709, stephan-stracke_csu                               

Following questions are not answered:
stephan-stracke_csu_Q0015_2018-11-11, stephan-stracke_

676 of 709, dr-katarina-barley_spd                            

Following questions are not answered:
dr-katarina-barley_spd_Q0017_2018-03-23, dr-katarina-barley_spd_Q0018_2018-03-28, dr-katarina-barley_spd_Q0037_2018-10-19, dr-katarina-barley_spd_Q0042_2018-11-20, dr-katarina-barley_spd_Q0043_2018-11-21, dr-katarina-barley_spd_Q0044_2018-11-21, dr-katarina-barley_spd_Q0045_2018-11-22, dr-katarina-barley_spd_Q0046_2018-11-28, dr-katarina-barley_spd_Q0047_2018-12-12, dr-katarina-barley_spd_Q0048_2018-12-12, dr-katarina-barley_spd_Q0049_2018-12-12, dr-katarina-barley_spd_Q0050_2018-12-12, dr-katarina-barley_spd_Q0051_2018-12-12, dr-katarina-barley_spd_Q0052_2018-12-12, dr-katarina-barley_spd_Q0053_2018-12-12, dr-katarina-barley_spd_Q0054_2018-12-12, dr-katarina-barley_spd_Q0055_2018-12-12, dr-katarina-barley_spd_Q0056_2018-12-12, dr-katarina-barley_spd_Q0057_2018-12-12, dr-katarina-barley_spd_Q0058_2018-12-12, dr-katarina-barley_spd_Q0059_2018-12-12, dr-katarina-barley_spd_Q0060_2018-12-

707 of 709, markus-kurth_die-grünen                           

Following questions are not answered:
markus-kurth_die-grünen_Q0005_2018-06-01, markus-kurth_die-grünen_Q0006_2018-09-09, markus-kurth_die-grünen_Q0008_2018-11-11

708 of 709, renate-kunast_die-grünen                          

Following questions are not answered:
renate-kunast_die-grünen_Q0030_2018-08-23, renate-kunast_die-grünen_Q0031_2018-11-14, renate-kunast_die-grünen_Q0032_2018-12-01, renate-kunast_die-grünen_Q0033_2018-12-04

709 of 709, britta-katharina-dassler_fdp                      

Following questions are not answered:
britta-katharina-dassler_fdp_Q0001_2017-08-01, britta-katharina-dassler_fdp_Q0002_2017-08-02, britta-katharina-dassler_fdp_Q0005_2018-08-13



<table style="width:100%">
  <tr>
      <td colspan="1" style="text-align:left;background-color:#0071BD;color:white">
        <a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/">
            <img alt="Creative Commons License" style="border-width:0;float:left;padding-right:10pt"
                 src="https://i.creativecommons.org/l/by-nc/4.0/88x31.png" />
        </a>
        &copy; D. Speicher<br/>
        Licensed under a 
        <a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/" style="color:white">
            CC BY-NC 4.0
        </a>.
      </td>
      <td colspan="2" style="text-align:left;background-color:#66A5D1">
          <b>Acknowledgments:</b>
          This material was prepared within the project
          <a href="http://www.b-it-center.de/b-it-programmes/teaching-material/p3ml/" style="color:black">
              P3ML
          </a> 
          which is funded by the Ministry of Education and Research of Germany (BMBF)
          under grant number 01/S17064. The authors gratefully acknowledge this support.
      </td>
  </tr>
</table>